# Routing Workflow

In [1]:
%useLatestDescriptors
%use coroutines
@file:DependsOn("dev.langchain4j:langchain4j:1.0.0-beta2")
@file:DependsOn("dev.langchain4j:langchain4j-open-ai:1.0.0-beta2")

In [2]:
val apiKey = System.getenv("OPENAI_API_KEY")

In [3]:
import dev.langchain4j.model.openai.OpenAiChatModel
import dev.langchain4j.model.openai.OpenAiChatModelName

val clientBuilder = OpenAiChatModel.builder()
    .apiKey(apiKey)
    .modelName(OpenAiChatModelName.GPT_4_O_MINI)
    .maxTokens(4096)
    .temperature(0.1)

In [4]:
import kotlinx.serialization.Serializable

/**
 * Class representing the response from the routing classification process.
 *
 * @property reasoning The reasoning behind the route selection, explaining why this particular
 * route was chosen based on the input analysis.
 * @property selection The selected route name that will handle the input based on the
 * classification analysis.
 */
@Serializable
data class RoutingResponse(val reasoning: String, val selection: String)

In [5]:
import dev.langchain4j.data.message.UserMessage
import dev.langchain4j.model.chat.Capability
import dev.langchain4j.model.chat.ChatLanguageModel
import dev.langchain4j.model.chat.request.ChatRequest
import dev.langchain4j.model.chat.request.ResponseFormat
import dev.langchain4j.model.chat.request.ResponseFormatType
import dev.langchain4j.model.chat.request.json.JsonObjectSchema
import dev.langchain4j.model.chat.request.json.JsonSchema
import kotlinx.serialization.json.Json

fun OpenAiChatModel.OpenAiChatModelBuilder.route(input: String, routes: Map<String, String>): String {
    val routeKey = determineRoute(input, routes)

    val client = this.build()
    val selectedPrompt = routes[routeKey]
    return client.chat("$selectedPrompt\n\nInput: $input")
}

fun OpenAiChatModel.OpenAiChatModelBuilder.determineRoute(input: String, routes: Map<String, String>): String {
    println("Available routes: ${routes.keys}")
    val selectorPrompt =
        """
        Analyze the input and select the most appropriate support team from these options: ${routes.keys}
        First explain your reasoning, then provide your selection in this JSON format:

        ```json
        {
            "reasoning": "Brief explanation of why this ticket should be routed to a specific team.
        Consider key terms, user intent, and urgency level.",
            "selection": "The chosen team name"
        }
        ```

        Input: $input
        """.trimIndent()


    val chatRequest = ChatRequest.builder()
        .messages(UserMessage(selectorPrompt))
        .responseFormat(
            ResponseFormat.builder()
                .type(ResponseFormatType.JSON)
                .jsonSchema(
                    JsonSchema.builder()
                        .name("RoutingResponse")
                        .rootElement(
                            JsonObjectSchema.builder()
                                .addStringProperty("reasoning")
                                .addStringProperty("selection")
                                .build()
                        )
                        .build()
                )
                .build()
        )
        .build()

    val client = this
        .supportedCapabilities(Capability.RESPONSE_FORMAT_JSON_SCHEMA)
        .strictJsonSchema(true)
        .build()

    val routingResponse = Json.decodeFromString<RoutingResponse>(client.chat(chatRequest).aiMessage().text())
    val (reasoning, routeKey) = routingResponse

    println("Routing Analysis:")
    println(reasoning)
    println("Selected route: $routeKey")

    return routeKey
}

In [6]:
val supportRoutes = mapOf(
    "billing" to """You are a billing support specialist. Follow these guidelines:
    1. Always start with "Billing Support Response:"
    2. First acknowledge the specific billing issue
    3. Explain any charges or discrepancies clearly
    4. List concrete next steps with timeline
    5. End with payment options if relevant

    Keep responses professional but friendly.

    Input: """,

    "technical" to """You are a technical support engineer. Follow these guidelines:
    1. Always start with "Technical Support Response:"
    2. List exact steps to resolve the issue
    3. Include system requirements if relevant
    4. Provide workarounds for common problems
    5. End with escalation path if needed

    Use clear, numbered steps and technical details.

    Input: """,

    "account" to """You are an account security specialist. Follow these guidelines:
    1. Always start with "Account Support Response:"
    2. Prioritize account security and verification
    3. Provide clear steps for account recovery/changes
    4. Include security tips and warnings
    5. Set clear expectations for resolution time

    Maintain a serious, security-focused tone.

    Input: """,

    "product" to """You are a product specialist. Follow these guidelines:
    1. Always start with "Product Support Response:"
    2. Focus on feature education and best practices
    3. Include specific examples of usage
    4. Link to relevant documentation sections
    5. Suggest related features that might help

    Be educational and encouraging in tone.

    Input: """
)

In [7]:
val tickets = listOf(
    """
    Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error.
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to
    submit a report by end of day.
    - John
    """,

    """
    Subject: Unexpected charge on my card
    Message: Hello, I just noticed a charge of $49.99 on my credit card from your company, but I thought
    I was on the $29.99 plan. Can you explain this charge and adjust it if it's a mistake?
    Thanks,
    Sarah
    """,

    """
    Subject: How to export data?
    Message: I need to export all my project data to Excel. I've looked through the docs but can't
    figure out how to do a bulk export. Is this possible? If so, could you walk me through the steps?
    Best regards,
    Mike
    """
)

In [8]:
println("Processing support tickets...")
tickets.forEachIndexed { index, ticket ->
    println("Ticket ${index + 1}:")
    println("-".repeat(40))
    println(ticket)
    println("Response:")
    println("-".repeat(40))
    val response = clientBuilder.route(ticket, supportRoutes)
    println(response)
}

Processing support tickets...
Ticket 1:
----------------------------------------

    Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error.
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to
    submit a report by end of day.
    - John
    
Response:
----------------------------------------
Available routes: [billing, technical, account, product]
Routing Analysis:
The user is experiencing an issue with logging into their account, specifically receiving an 'invalid password' error despite using what they believe is the correct password. This indicates a problem related to account access rather than billing, technical issues with the product itself, or inquiries about product features. Given the urgency expressed by the user, as they need to regain access to submit a report by the end of the day, the account support team is best suited to assist with 